In [40]:
# instalar todas as libs do requirements.txt
%pip freeze -r requirements.txt

aiohttp==3.11.16
aiosignal==1.3.2
annotated-types==0.7.0
anyio==4.9.0
asttokens==3.0.0
attrs==25.3.0
certifi==2025.1.31
charset-normalizer==3.4.1
comm==0.2.2
datasets==2.15.0
debugpy==1.8.14
decorator==5.2.1
dill==0.3.7
distro==1.9.0
executing==2.2.0
filelock==3.18.0
frozenlist==1.5.0
fsspec==2023.10.0
h11==0.14.0
httpcore==1.0.8
httpx==0.28.1
huggingface-hub==0.30.2
idna==3.10
ipykernel==6.29.5
ipython==9.1.0
jedi==0.19.2
jiter==0.9.0
jupyter_client==8.6.3
jupyter_core==5.7.2
matplotlib-inline==0.1.7
multidict==6.4.3
multiprocess==0.70.15
nest-asyncio==1.6.0
numpy==2.2.4
openai==1.74.0
packaging==24.2
pandas==2.2.3
parso==0.8.4
pexpect==4.9.0
platformdirs==4.3.7
prompt_toolkit==3.0.51
psutil==7.0.0
ptyprocess==0.7.0
pure_eval==0.2.3
pyarrow==19.0.1
pyarrow-hotfix==0.6
pydantic==2.11.3
pydantic_core==2.33.1
Pygments==2.19.1
python-dateutil==2.9.0.post0
pytz==2025.2
PyYAML==6.0.2
pyzmq==26.4.0
requests==2.32.3
six==1.17.0
sniffio==1.3.1
stack-data==0.6.3
tornado==6.4.2
tqdm==4.67.1
trai

In [41]:
# instalar o datasets da huggingface
%pip install datasets==2.15.0

Note: you may need to restart the kernel to use updated packages.


In [42]:
import json
from datasets import load_dataset

In [43]:
datasets = load_dataset('hate-speech-portuguese/hate_speech_portuguese',split='train[:10%]')

In [30]:
# print dataset
print(datasets)

Dataset({
    features: ['text', 'label', 'hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'],
    num_rows: 567
})


In [44]:
datasets = datasets.remove_columns(['hatespeech_G1', 'annotator_G1', 'hatespeech_G2', 'annotator_G2', 'hatespeech_G3', 'annotator_G3'])

print(datasets)

Dataset({
    features: ['text', 'label'],
    num_rows: 567
})


In [45]:
datasets = datasets.train_test_split(test_size=0.2) # método para testar o dataset

In [46]:
# removar os /n das mensagems

def removeN(example):
    example['text'] = example['text'].replace('\n', '')
    return example

# aplicando a função para cada exemplo do dataset

datasets = datasets.map(removeN)

datasets

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map: 100%|██████████| 114/114 [00:00<00:00, 6020.30 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 453
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 114
    })
})

In [47]:
# label 0 = No Hate Speech
# label 1 = Hate Speech

def labelChange(example):
    example['label_text'] = 'No Hate Speech' if example['label'] == 0 else 'Hate Speech' 
    return example

In [48]:
# aplicando a função para cada exemplo do dataset
datasets = datasets.map(labelChange)

# removendo a coluna label
datasets = datasets.remove_columns(['label'])

Map: 100%|██████████| 114/114 [00:00<00:00, 7099.07 examples/s]


In [49]:
# Adequar o dataset para o formato do modelo

def dataset_to_json(dataset, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for example in dataset:
            json_obj = { "messages": [
               { "role": "system", "content":"Seu trabalho é classificar os comentários dos usuários em Hate Speech ou No Hate Speech"},
               { "role": "user", "content": example['text']},
               { "role": "assistant", "content": example['label_text']}
              ]
            }
            f.write(json.dumps(json_obj, ensure_ascii=False)+"\n")

In [37]:
# construção o arquivo jsonl
dataset_to_json(datasets['train'], 'train.jsonl')

In [50]:
# criação do arquivo de validação (output)
dataset_to_json(datasets['test'], 'validation.jsonl')

In [51]:
# fine-tunning job
from openai import OpenAI
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = 'sk-...'

In [ ]:
client = OpenAI()

In [ ]:
# upload do arquivo de treino

trainingFile = client.files.create(
    file=open('train.jsonl', 'rb'),
    purpose='fine-tune'
)

In [ ]:
# upload do arquivo de validação

validationFile = client.files.create(
    file=open('validation.jsonl', 'rb'),
    purpose='fine-tune'
)

In [ ]:
client.fine_tunning.create(
    training_file=trainingFile['id'],
    validation_file=validationFile['id'],
    model="gpt-3.5-turbo"
)